In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as its
import function_pontius as fpt
import heatmap_cm as hmcm
from pathlib import Path
import openpyxl
import xlsxwriter

# Compare maps with ponti DF

In [54]:
metric = 'kappa'

def heatmap_cm(map1, map2, metric=metric):
    """This function takes in two maps in array form and returns a heatmap CM
    """
    # Load in the maps and set up variables to process the maps
    map1_dir = 'E:/thesis_data/asc_2k/' + map1 + '.asc'
    map2_dir = 'E:/thesis_data/asc_2k/' + map2 + '.asc'    
    m1 = np.loadtxt(map1_dir, skiprows=6)
    m2 = np.loadtxt(map2_dir, skiprows=6)
    cm = fpt.conf_mat(m1, m2)
    # Set up dataframe and adjust for land-use categories that are removed from the df
    rows_cols = ['class' + str(i + 1) for i in range(28)]
    rows_cols.remove('class24')
    rows_cols.remove('class28')
    df = pd.DataFrame(data=cm, index=rows_cols, columns=rows_cols)
    # Create the pontius class and generate the cm matrix with additional info
    df_pont = hmcm.pontiPy(df)
    df_matrix = df_pont.matrix()
    df_matrix['RowDisagreement'] = pd.to_numeric(df_matrix['RowDisagreement'])
    # Assign dummy value to cell to prevent error warning
    df_matrix.loc['ColumnDisagreement', 'RowDisagreement'] = 0
    # Apply heatmap to cm using pandas styler
    df_heatmap = df_matrix.style.background_gradient(cmap='rocket')
    #Store data in excel sheet with other representative maps of the metric
    path = 'E:/thesis_images/representative_maps/excels/' + metric + '.xlsx'
    file = Path(path)
    # If no excel yet create one
    if not file.is_file():
        wb = xlsxwriter.Workbook(path)
        wb.close()
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    book = openpyxl.load_workbook(path)
    writer.book = book
    # Write each map pair to their own sheet
    sheet_id = map1 + '_' + map2
    df_heatmap.to_excel(writer, sheet_name=sheet_id)
    writer.save()
    writer.close()

In [53]:
#Save images to disk of graphs displaying difference between representative maps for kappa
kappa_maps = ['map359', 'map456', 'map541', 'map695', 'map1148', 'map1760', 'map1911']
iterator = its.combinations(kappa_maps, 2)

for y in its.starmap(heatmap_cm, iterator):
    pass

In [55]:
# Repeat for OA
metric = 'oa'
oa_maps = ['map10', 'map359', 'map456', 'map541', 'map695', 'map696', 'map1911']
iterator = its.combinations(oa_maps, 2)

for y in its.starmap(heatmap_cm, iterator):
    pass

# One CM that combines disagreement rows for all maps (and columns in another one)

In [80]:
data = []
row_name = []

def heatmap_rowdisagreement(map1, map2, rowcol='row'):
    """This function takes in two map names and returns the rowdisagreement from their confusion matrix.
    The returned values show for each class the percentage (relative to the whole map) of cells that map 1 
    predicted differently than map 2 for that class.
    """
    # Load in the maps and set up variables to process the maps
    map1_dir = 'E:/thesis_data/asc_2k/' + map1 + '.asc'
    map2_dir = 'E:/thesis_data/asc_2k/' + map2 + '.asc'    
    m1 = np.loadtxt(map1_dir, skiprows=6)
    m2 = np.loadtxt(map2_dir, skiprows=6)
    cm = fpt.conf_mat(m1, m2)
    # Set up dataframe and adjust for land-use categories that are removed from the df
    rows_cols = ['class' + str(i + 1) for i in range(28)]
    rows_cols.remove('class24')
    rows_cols.remove('class28')
    df = pd.DataFrame(data=cm, index=rows_cols, columns=rows_cols)
    # Create the pontius class and generate the cm matrix with additional info
    df_pont = hmcm.pontiPy(df)
    df_matrix = df_pont.matrix()
    df_matrix['RowDisagreement'] = pd.to_numeric(df_matrix['RowDisagreement'])
    # Assign dummy value to cell to prevent error warning
    df_matrix.loc['ColumnDisagreement', 'RowDisagreement'] = 0
    #Store disagreement
    data = df_matrix.loc[:'class27', 'RowDisagreement']
    # Create label for new df
    row_id = map1 + '_' + map2
    return data, row_id

def heatmap_coldisagreement(map1, map2):
    """This function takes in two map names and returns the coldisagreement from their confusion matrix.
    The returned values show for each class the percentage (relative to the whole map) of cells that map 2 
    predicted differently than map 1 for that class.
    """
    # Load in the maps and set up variables to process the maps
    map1_dir = 'E:/thesis_data/asc_2k/' + map1 + '.asc'
    map2_dir = 'E:/thesis_data/asc_2k/' + map2 + '.asc'    
    m1 = np.loadtxt(map1_dir, skiprows=6)
    m2 = np.loadtxt(map2_dir, skiprows=6)
    cm = fpt.conf_mat(m1, m2)
    # Set up dataframe and adjust for land-use categories that are removed from the df
    rows_cols = ['class' + str(i + 1) for i in range(28)]
    rows_cols.remove('class24')
    rows_cols.remove('class28')
    df = pd.DataFrame(data=cm, index=rows_cols, columns=rows_cols)
    # Create the pontius class and generate the cm matrix with additional info
    df_pont = hmcm.pontiPy(df)
    df_matrix = df_pont.matrix()
    df_matrix['RowDisagreement'] = pd.to_numeric(df_matrix['RowDisagreement'])
    # Assign dummy value to cell to prevent error warning
    df_matrix.loc['ColumnDisagreement', 'RowDisagreement'] = 0
    #Store disagreement
    data = df_matrix.loc['ColumnDisagreement', :'class27']    
    # Create label for new df
    row_id = map1 + '_' + map2
    return data, row_id
    
def heatmap_to_excel(hm, metric, rowcol='row'):
    if rowcol == 'row':
        excel_dir = 'E:/thesis_images/representative_maps/excels/heatmap_rows.xlsx'
    else:
        excel_dir = 'E:/thesis_images/representative_maps/excels/heatmap_cols.xlsx'
    
    writer = pd.ExcelWriter(excel_dir, engine = 'openpyxl')
    book = openpyxl.load_workbook(excel_dir)
    writer.book = book
    # Write each metric to their own sheet
    sheet_id = metric
    hm.to_excel(writer, sheet_name=sheet_id)

    writer.save()
    writer.close()

In [26]:
# Create CM showing rowdisagreement for kappa representative maps
# Overal difference has same maps as kappa so no point 
kappa_data = []
kappa_row_name = []
kappa_maps = ['map359', 'map456', 'map541', 'map695', 'map1148', 'map1760', 'map1911']
iterator = its.combinations(kappa_maps, 2)

for y in its.starmap(heatmap_rowdisagreement, iterator):
    pass

df = pd.DataFrame(data=kappa_data, index=kappa_row_name)
# Apply heatmap to cm using pandas styler
kappa_heatmap = df.style.background_gradient(cmap='rocket')

In [44]:
# Create CM showing rowdisagreement for oa representative maps
data = []
row_name = []
oa_maps = ['map10', 'map359', 'map456', 'map541', 'map695', 'map696', 'map1911']
iterator = its.combinations(oa_maps, 2)

for y in its.starmap(heatmap_rowdisagreement, iterator):
    pass

df2 = pd.DataFrame(data=data, index=row_name)
# Apply heatmap to cm using pandas styler
oa_heatmap = df2.style.background_gradient(cmap='rocket')

In [60]:
# Create CM showing rowdisagreement for shannon's representative maps
shan_data = []
shan_row_name = []
shan_maps = ['map61', 'map1248', 'map1454', 'map1457']
iterator = its.combinations(shan_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    shan_data.append(dat)
    shan_row_name.append(row)

df3 = pd.DataFrame(data=shan_data, index=shan_row_name)
# Apply heatmap to cm using pandas styler
shan_heatmap = df3.style.background_gradient(cmap='rocket')

In [61]:
# Create CM showing rowdisagreement for simpson's representative maps
simp_data = []
simp_row_name = []
simp_maps = ['map65', 'map857', 'map1448', 'map1784', 'map1830']
iterator = its.combinations(simp_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    simp_data.append(dat)
    simp_row_name.append(row)
    
df4 = pd.DataFrame(data=simp_data, index=simp_row_name)
# Apply heatmap to cm using pandas styler
simp_heatmap = df4.style.background_gradient(cmap='rocket')

In [64]:
# Create CM showing rowdisagreement for oad representative maps
oad_data = []
oad_row_name = []
oad_maps = ['map263', 'map453', 'map678', 'map1760', 'map1852', 'map1918']
iterator = its.combinations(oad_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    oad_data.append(dat)
    oad_row_name.append(row)
    
df5 = pd.DataFrame(data=oad_data, index=oad_row_name)
# Apply heatmap to cm using pandas styler
oad_heatmap = df5.style.background_gradient(cmap='rocket')

In [65]:
# Create CM showing rowdisagreement for oqd representative maps
oqd_data = []
oqd_row_name = []
oqd_maps = ['map1057', 'map1472', 'map1833', 'map1978']
iterator = its.combinations(oqd_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    oqd_data.append(dat)
    oqd_row_name.append(row)
    
df6 = pd.DataFrame(data=oqd_data, index=oqd_row_name)
# Apply heatmap to cm using pandas styler
oqd_heatmap = df6.style.background_gradient(cmap='rocket')

In [66]:
# Create CM showing rowdisagreement for tca4 representative maps
tca4_data = []
tca4_row_name = []
tca4_maps = ['map217', 'map332', 'map604']
iterator = its.combinations(tca4_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    tca4_data.append(dat)
    tca4_row_name.append(row)
    
df7 = pd.DataFrame(data=oqd_data, index=oqd_row_name)
# Apply heatmap to cm using pandas styler
tca4_heatmap = df7.style.background_gradient(cmap='rocket')

In [71]:
# Create CM showing rowdisagreement for tca4 representative maps
pland4_data = []
pland4_row_name = []
pland4_maps = ['map264', 'map1422', 'map1877']
iterator = its.combinations(pland4_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    pland4_data.append(dat)
    pland4_row_name.append(row)
    
df8 = pd.DataFrame(data=pland4_data, index=pland4_row_name)
# Apply heatmap to cm using pandas styler
pland4_heatmap = df8.style.background_gradient(cmap='rocket')

In [70]:
# Create CM showing rowdisagreement for tca4 representative maps
tca14_data = []
tca14_row_name = []
tca14_maps = ['map246', 'map296', 'map325', 'map825', 'map1612']
iterator = its.combinations(tca14_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    tca14_data.append(dat)
    tca14_row_name.append(row)
    
df9 = pd.DataFrame(data=tca14_data, index=tca14_row_name)
# Apply heatmap to cm using pandas styler
tca14_heatmap = df9.style.background_gradient(cmap='rocket')

In [72]:
# Create CM showing rowdisagreement for tca4 representative maps
pland14_data = []
pland14_row_name = []
pland14_maps = ['map246', 'map325', 'map564', 'map704', 'map843']
iterator = its.combinations(pland14_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    pland14_data.append(dat)
    pland14_row_name.append(row)
    
df10 = pd.DataFrame(data=pland14_data, index=pland14_row_name)
# Apply heatmap to cm using pandas styler
pland14_heatmap = df10.style.background_gradient(cmap='rocket')

In [73]:
# Create CM showing rowdisagreement for tca4 representative maps
# TCA22 and PLAND 22 have the same representative maps so only TCA will be done for both of them
tca22_data = []
tca22_row_name = []
tca22_maps = ['map0', 'map8', 'map35']
iterator = its.combinations(tca22_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    tca22_data.append(dat)
    tca22_row_name.append(row)
    
df11 = pd.DataFrame(data=tca22_data, index=tca22_row_name)
# Apply heatmap to cm using pandas styler
tca22_heatmap = df11.style.background_gradient(cmap='rocket')

In [74]:
# Create CM showing rowdisagreement for qd22 representative maps
qd22_data = []
qd22_row_name = []
qd22_maps = ['map0', 'map1', 'map8']
iterator = its.combinations(qd22_maps, 2)

for dat, row in its.starmap(heatmap_rowdisagreement, iterator):
    qd22_data.append(dat)
    qd22_row_name.append(row)
    
df12 = pd.DataFrame(data=qd22_data, index=qd22_row_name)
# Apply heatmap to cm using pandas styler
qd22_heatmap = df12.style.background_gradient(cmap='rocket')

In [75]:
# Store all heatmaps in an excel
heatmap_to_excel(kappa_heatmap, 'kappa', rowcol='row')
heatmap_to_excel(oa_heatmap, 'oa', rowcol='row')
heatmap_to_excel(shan_heatmap, 'shan', rowcol='row')
heatmap_to_excel(simp_heatmap, 'simp', rowcol='row')
heatmap_to_excel(oad_heatmap, 'oad', rowcol='row')
heatmap_to_excel(oqd_heatmap, 'oqd', rowcol='row')
heatmap_to_excel(tca4_heatmap, 'tca4', rowcol='row')
heatmap_to_excel(pland4_heatmap, 'pland4', rowcol='row')
heatmap_to_excel(tca14_heatmap, 'tca14', rowcol='row')
heatmap_to_excel(pland14_heatmap, 'pland14', rowcol='row')
heatmap_to_excel(tca22_heatmap, 'tca22', rowcol='row')
heatmap_to_excel(qd22_heatmap, 'qd22', rowcol='row')

## Repeat now for col disagreement

In [86]:
# Create CM showing rcoldisagreement for kappa representative maps
# Overal difference has same maps as kappa so no point doing both
kappa_data = []
kappa_row_name = []
kappa_maps = ['map359', 'map456', 'map541', 'map695', 'map1148', 'map1760', 'map1911']
iterator = its.combinations(kappa_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    kappa_data.append(dat)
    kappa_row_name.append(row)

df = pd.DataFrame(data=kappa_data, index=kappa_row_name)
# Apply heatmap to cm using pandas styler
kappa_heatmap = df.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for oa representative maps
oa_data = []
oa_row_name = []
oa_maps = ['map10', 'map359', 'map456', 'map541', 'map695', 'map696', 'map1911']
iterator = its.combinations(oa_maps, 2)

for dat,row in its.starmap(heatmap_coldisagreement, iterator):
    oa_data.append(dat)
    oa_row_name.append(row)

df2 = pd.DataFrame(data=oa_data, index=oa_row_name)
# Apply heatmap to cm using pandas styler
oa_heatmap = df2.style.background_gradient(cmap='rocket')

# Create CM showing coldisagreement for shannon's representative maps
shan_data = []
shan_row_name = []
shan_maps = ['map61', 'map1248', 'map1454', 'map1457']
iterator = its.combinations(shan_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    shan_data.append(dat)
    shan_row_name.append(row)

df3 = pd.DataFrame(data=shan_data, index=shan_row_name)
# Apply heatmap to cm using pandas styler
shan_heatmap = df3.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for simpson's representative maps
simp_data = []
simp_row_name = []
simp_maps = ['map65', 'map857', 'map1448', 'map1784', 'map1830']
iterator = its.combinations(simp_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    simp_data.append(dat)
    simp_row_name.append(row)
    
df4 = pd.DataFrame(data=simp_data, index=simp_row_name)
# Apply heatmap to cm using pandas styler
simp_heatmap = df4.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for oad representative maps
oad_data = []
oad_row_name = []
oad_maps = ['map263', 'map453', 'map678', 'map1760', 'map1852', 'map1918']
iterator = its.combinations(oad_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    oad_data.append(dat)
    oad_row_name.append(row)
    
df5 = pd.DataFrame(data=oad_data, index=oad_row_name)
# Apply heatmap to cm using pandas styler
oad_heatmap = df5.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for oqd representative maps
oqd_data = []
oqd_row_name = []
oqd_maps = ['map1057', 'map1472', 'map1833', 'map1978']
iterator = its.combinations(oqd_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    oqd_data.append(dat)
    oqd_row_name.append(row)
    
df6 = pd.DataFrame(data=oqd_data, index=oqd_row_name)
# Apply heatmap to cm using pandas styler
oqd_heatmap = df6.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for tca4 representative maps
tca4_data = []
tca4_row_name = []
tca4_maps = ['map217', 'map332', 'map604']
iterator = its.combinations(tca4_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    tca4_data.append(dat)
    tca4_row_name.append(row)
    
df7 = pd.DataFrame(data=oqd_data, index=oqd_row_name)
# Apply heatmap to cm using pandas styler
tca4_heatmap = df7.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for tca4 representative maps
pland4_data = []
pland4_row_name = []
pland4_maps = ['map264', 'map1422', 'map1877']
iterator = its.combinations(pland4_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    pland4_data.append(dat)
    pland4_row_name.append(row)
    
df8 = pd.DataFrame(data=pland4_data, index=pland4_row_name)
# Apply heatmap to cm using pandas styler
pland4_heatmap = df8.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for tca4 representative maps
tca14_data = []
tca14_row_name = []
tca14_maps = ['map246', 'map296', 'map325', 'map825', 'map1612']
iterator = its.combinations(tca14_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    tca14_data.append(dat)
    tca14_row_name.append(row)
    
df9 = pd.DataFrame(data=tca14_data, index=tca14_row_name)
# Apply heatmap to cm using pandas styler
tca14_heatmap = df9.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for tca4 representative maps
pland14_data = []
pland14_row_name = []
pland14_maps = ['map246', 'map325', 'map564', 'map704', 'map843']
iterator = its.combinations(pland14_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    pland14_data.append(dat)
    pland14_row_name.append(row)
    
df10 = pd.DataFrame(data=pland14_data, index=pland14_row_name)
# Apply heatmap to cm using pandas styler
pland14_heatmap = df10.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for tca4 representative maps
# TCA22 and PLAND 22 have the same representative maps so only TCA will be done for both of them
tca22_data = []
tca22_row_name = []
tca22_maps = ['map0', 'map8', 'map35']
iterator = its.combinations(tca22_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    tca22_data.append(dat)
    tca22_row_name.append(row)
    
df11 = pd.DataFrame(data=tca22_data, index=tca22_row_name)
# Apply heatmap to cm using pandas styler
tca22_heatmap = df11.style.background_gradient(cmap='rocket')

# Create CM showing rowdisagreement for qd22 representative maps
qd22_data = []
qd22_row_name = []
qd22_maps = ['map0', 'map1', 'map8']
iterator = its.combinations(qd22_maps, 2)

for dat, row in its.starmap(heatmap_coldisagreement, iterator):
    qd22_data.append(dat)
    qd22_row_name.append(row)
    
df12 = pd.DataFrame(data=qd22_data, index=qd22_row_name)
# Apply heatmap to cm using pandas styler
qd22_heatmap = df12.style.background_gradient(cmap='rocket')

# Store all heatmaps in an excel
heatmap_to_excel(kappa_heatmap, 'kappa', rowcol='col')
heatmap_to_excel(oa_heatmap, 'oa', rowcol='col')
heatmap_to_excel(shan_heatmap, 'shan', rowcol='col')
heatmap_to_excel(simp_heatmap, 'simp', rowcol='col')
heatmap_to_excel(oad_heatmap, 'oad', rowcol='col')
heatmap_to_excel(oqd_heatmap, 'oqd', rowcol='col')
heatmap_to_excel(tca4_heatmap, 'tca4', rowcol='col')
heatmap_to_excel(pland4_heatmap, 'pland4', rowcol='col')
heatmap_to_excel(tca14_heatmap, 'tca14', rowcol='col')
heatmap_to_excel(pland14_heatmap, 'pland14', rowcol='col')
heatmap_to_excel(tca22_heatmap, 'tca22', rowcol='col')
heatmap_to_excel(qd22_heatmap, 'qd22', rowcol='col')

# test area

In [76]:
m1 = np.loadtxt('E:/thesis_data/asc_2k/map541.asc', skiprows=6)
m2 = np.loadtxt('E:/thesis_data/asc_2k/map695.asc', skiprows=6)
cm = fpt.conf_mat(m1, m2)
rows_cols = ['class' + str(i + 1) for i in range(28)]
rows_cols.remove('class24')
rows_cols.remove('class28')
df = pd.DataFrame(data=cm, index=rows_cols, columns=rows_cols)
# Create the pontius class and generate the cm matrix with additional info
df_pont = hmcm.pontiPy(df)
df_matrix = df_pont.matrix()
df_matrix['RowDisagreement'] = pd.to_numeric(df_matrix['RowDisagreement'])
# Assign dummy value to cell to prevent error warning
df_matrix.loc['ColumnDisagreement', 'RowDisagreement'] = 0

In [78]:
df_matrix.loc['ColumnDisagreement', :'class27']

class1     0.000000
class2     0.000000
class3     0.000000
class4     0.273331
class5     0.957510
class6     0.006046
class7     0.000000
class8     0.004993
class9     0.000000
class10    0.000756
class11    0.011849
class12    0.000000
class13    0.026666
class14    0.211011
class15    0.042051
class16    0.017355
class17    0.027017
class18    0.001134
class19    0.034494
class20    0.647795
class21    0.021484
class22    0.008313
class23    0.955620
class25    0.000000
class26    0.000000
class27    0.000000
Name: ColumnDisagreement, dtype: float64

In [79]:
df_matrix

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,...,class19,class20,class21,class22,class23,class25,class26,class27,Sum,RowDisagreement
class1,0.246071,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.246071,0.000000
class2,0.000000,2.913014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.913014,0.000000
class3,0.000000,0.000000,0.087152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.087152,0.000000
class4,0.000000,0.000000,0.000000,9.292804,0.092955,0.002780,0.000000,0.004264,0.000000,0.000162,...,0.001619,0.048205,0.000621,0.001430,0.006748,0.000000,0.000000,0.00000,9.497769,0.204965
class5,0.000000,0.000000,0.000000,0.002645,2.201684,0.000027,0.000000,0.000135,0.000000,0.000000,...,0.000000,0.000675,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.206785,0.005101
class6,0.000000,0.000000,0.000000,0.000216,0.005722,0.722397,0.000000,0.000027,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.728443,0.006046
class7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.391792,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.391792,0.000000
class8,0.000000,0.000000,0.000000,0.146989,0.015519,0.001943,0.000000,0.213710,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.002672,0.000000,0.000000,0.00000,0.381104,0.167394
class9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103103,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.103103,0.000000
class10,0.000000,0.000000,0.000000,0.000081,0.000162,0.000000,0.000000,0.000000,0.000000,0.313682,...,0.000162,0.000297,0.000027,0.000000,0.000027,0.000000,0.000000,0.00000,0.314438,0.000756


In [19]:
data.append(df_matrix.loc[:'class27', 'RowDisagreement'])

In [22]:
df = pd.DataFrame(data=data)

In [23]:
df

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,...,class17,class18,class19,class20,class21,class22,class23,class25,class26,class27
RowDisagreement,0.0,0.0,0.0,2.007866,0.188555,0.003752,0.0,0.014332,0.0,0.001403,...,0.063589,0.005290,0.038488,0.670305,0.013981,0.009690,1.281367,0.0,0.0,0.0
RowDisagreement,0.0,0.0,0.0,0.204965,0.005101,0.006046,0.0,0.167394,0.0,0.000756,...,0.138191,0.031552,0.034494,1.283904,0.021484,0.008313,0.955674,0.0,0.0,0.0


In [14]:
df_matrix

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,...,class19,class20,class21,class22,class23,class25,class26,class27,Sum,RowDisagreement
class1,0.246071,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.246071,0.000000
class2,0.000000,2.913014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.913014,0.000000
class3,0.000000,0.000000,0.087152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.087152,0.000000
class4,0.000000,0.000000,0.000000,9.568214,0.174331,0.003509,0.000000,0.047044,0.000000,0.001215,...,0.008502,0.499807,0.009878,0.005425,0.920533,0.000000,0.000000,0.00000,11.576080,2.007866
class5,0.000000,0.000000,0.000000,0.000351,3.213228,0.000216,0.000000,0.000621,0.000000,0.000000,...,0.005830,0.060890,0.000702,0.000999,0.084831,0.000000,0.000000,0.00000,3.401783,0.188555
class6,0.000000,0.000000,0.000000,0.000000,0.001754,0.724691,0.000000,0.000000,0.000000,0.000000,...,0.000027,0.000540,0.000000,0.000000,0.001242,0.000000,0.000000,0.00000,0.728443,0.003752
class7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.391792,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.391792,0.000000
class8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128150,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.014332,0.000000,0.000000,0.00000,0.142482,0.014332
class9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103103,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.103103,0.000000
class10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.313034,...,0.000000,0.000243,0.000081,0.000027,0.001053,0.000000,0.000000,0.00000,0.314438,0.001403
